## Main Pipeline
- 2 models were trained to predict the estimated E_A & A_0 via Arrhenius method. (LGBM only bc it was the best performer during coding experiments)
- Then, the main model (volatile release prediction) was trained by including and excluding the predicted Arrhenius parameters

The entire code could be converted into object-oriented via AI tools easily if working in this setting is more convenient for the user.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap
from shap import plots
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from scipy.stats import uniform, randint
import joblib
import os
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor

# ----------------------------------------------------------------------
# 1) LOAD & PREPROCESS
# ----------------------------------------------------------------------
data_path = r"C:\Users\demir\OneDrive\Desktop\MSc Thesis\lasstttrun\mad1_Ea.csv" #PATH HERE (cleaned dataset (via MAD) & Arrhenius parameters included)
df = pd.read_csv(data_path)
eps = 1e-6

df['devol_yield'] = df['devol_yield']/(100 - df['ac'])
df = df[(df['fuel_type'] != 'Torr-Wood')]

cat2_map = {
    'Digestate':             'Biomass',
    'Sewage':                'Mix',
    'HTC-MSW':               'Mix',
    'Cellulose':             'Biomass',
    'Lignin':                'Biomass',
    'Hemicellulose':         'Biomass',
    'Wood':                  'Biomass',
    'Torr-Wood':             'Biomass',
    'Lignite':               'Coal',
    'Torr-Coal':             'Coal',
    'Rubber':                'Plastic',
    'RDF1':                  'Plastic',
    'RDF2':                  'Mix',
    'Digestate_PP':          'Co-Pyro',
    'Digestate_SCP':         'Co-Pyro',
    'Digestate_PE':          'Co-Pyro'
}



therm_map = {
    'Digestate':             'Yes',
    'Sewage':                'No',
    'HTC-MSW':               'Yes',
    'Cellulose':             'No',
    'Lignin':                'Yes',
    'Hemicellulose':         'No',
    'Wood':                  'No',
    'Torr-Wood':             'Yes',
    'Lignite':               'No',
    'Torr-Coal':             'Yes',
    'Rubber':                'No',
    'RDF1':                  'No',
    'RDF2':                  'No',
    'Digestate_PP':          'No',
    'Digestate_SCP':         'No',
    'Digestate_PE':          'No'
}


df['fuel_category2']   = df['fuel_type'].map(cat2_map).fillna('Unknown')
df['thermal_treatment'] = df['fuel_type'].map(therm_map).fillna('Unknown')

# One‐hot encode
df = pd.get_dummies(
    df,
    columns=['fuel_category2', 'thermal_treatment'],
    prefix=['fuel_cat2', 'th_treat'],
    drop_first=False
)

# ratio features
df['vm_fc']  = df['vm']         / (df['fc'] + eps)
df['oh_w'] = df['o'] / (df['h'] + eps)
df['ac_fc']  = df['ac']         / (df['fc'] + eps)
df['t_to_T'] = (df['temperature'] + 273.15) / df['heat_rate']
df['cl_ac']  = df['cl']         / (df['ac'] + eps)
df['n_ac']   = df['n']          / (df['ac'] + eps)

# ----------------------------------------------------------------------
# ——— BASIC LOG TRANSFORMS ———
for col in ['vm_fc', 'ac_fc', 'cl_ac', 'n_ac', 't_to_T','oc', 'hc', 'heat_rate', 'pressure']:
    df[f'log_{col}'] = np.log(df[col] + eps)


# ----------------------------------------------------------------------
# STEP-1: Ea FIT WITH HYPERPARAMETER TUNING
# ----------------------------------------------------------------------
# define features to use for Ea fitting
ea_input_features = ['hc','oc', 'vm_fc', 'temperature', 'heat_rate',
                     'residence_time', 'pressure',
                      'ac_fc', 't_to_T',
                     'fuel_cat2_Biomass', 'fuel_cat2_Coal', 'fuel_cat2_Mix',
                     'fuel_cat2_Plastic', 'th_treat_No', 'th_treat_Yes']
X_ea = df[[col for col in ea_input_features if col in df.columns]]
y_ea = df['E_A']
groups = df['fuel_type']

# hyperparameter ranges for LGBM
param_space_ea = {
    "model__n_estimators":     randint(100, 500),
    "model__learning_rate":    uniform(0.01, 0.3),
    "model__num_leaves":       randint(20, 100),
    "model__max_depth":        randint(3, 10),
    "model__subsample":        uniform(0.6, 0.4),
    "model__colsample_bytree": uniform(0.6, 0.4),
    "model__min_child_samples":randint(5, 50),
}

# pipeline + randomized search
pipe_ea = Pipeline([
    ("scaler", StandardScaler()),
    ("model", lgb.LGBMRegressor(objective="huber", random_state=42))
])

gkf = GroupKFold(n_splits=len(groups.unique()))
search_ea = RandomizedSearchCV(
    pipe_ea,
    param_distributions=param_space_ea,
    n_iter=30,
    cv=gkf,
    scoring="neg_mean_squared_error",
    n_jobs=-1,
    random_state=42,
    verbose=1
)

# fit to find best Ea hyperparameters
search_ea.fit(X_ea, y_ea, groups=groups)
best_params_ea = search_ea.best_params_
print(">> Best Ea‐model params:", best_params_ea)

# leave-one-fuel-out evaluation of the tuned Ea model
ea_metrics = []
df['Ea_pred_tuned'] = np.nan

for fuel in groups.unique():
    tr = groups != fuel
    te = ~tr

    Xtr, ytr = X_ea[tr], y_ea[tr]
    Xte, yte = X_ea[te], y_ea[te]

    # scale
    scaler = StandardScaler().fit(Xtr)
    Xtr_s, Xte_s = scaler.transform(Xtr), scaler.transform(Xte)

    tuned_ea = lgb.LGBMRegressor(
        objective="huber",
        random_state=42,
        **{k.split("__",1)[1]: v for k, v in best_params_ea.items()}
    )
    tuned_ea.fit(Xtr_s, ytr)

    ypred = tuned_ea.predict(Xte_s)
    ea_metrics.append({
        "Fuel": fuel,
        "R2":   r2_score(yte, ypred),
        "MAE":  mean_absolute_error(yte, ypred),
        "MSE": mean_squared_error(yte, ypred),
    })

    df.loc[te, 'Ea_pred_tuned'] = ypred


scaler_ea_all = StandardScaler().fit(X_ea)
X_ea_all_scaled = scaler_ea_all.transform(X_ea)

final_ea_model = lgb.LGBMRegressor(
    objective="huber",
    random_state=42,
    **{k.split("__", 1)[1]: v for k, v in best_params_ea.items()}
)
final_ea_model.fit(X_ea_all_scaled, y_ea)

#joblib.dump((final_ea_model, scaler_ea_all), PATH HERE")

# display Ea tuning results
ea_results_df = pd.DataFrame(ea_metrics)
print("\nTuned Ea LOFO results:")
print(ea_results_df)

# use the tuned predictions going forward
df['EA_Arr'] = df['Ea_pred_tuned']

# ----------------------------------------------------------------------
# 2) STEP-1: A0 FIT WITH HYPERPARAMETER TUNING
# ----------------------------------------------------------------------
# define features to use for A0 fitting
A0_input_features = [
    'hc','oc', 'vm_fc', 'temperature', 'heat_rate',
                     'residence_time', 'pressure',
                      'ac_fc', 't_to_T',
                     'fuel_cat2_Biomass', 'fuel_cat2_Coal', 'fuel_cat2_Mix',
                     'fuel_cat2_Plastic', 'th_treat_No', 'th_treat_Yes'
]

X_A = df[[col for col in A0_input_features if col in df.columns]]
y_A = df['A0']
groups = df['fuel_type']

# hyperparameter ranges for LGBM
param_space_A = {
    "model__n_estimators":     randint(100, 500),
    "model__learning_rate":    uniform(0.01, 0.3),
    "model__num_leaves":       randint(20, 100),
    "model__max_depth":        randint(3, 10),
    "model__subsample":        uniform(0.6, 0.4),
    "model__colsample_bytree": uniform(0.6, 0.4),
    "model__min_child_samples":randint(5, 50),
}

# pipeline + randomized search
pipe_A = Pipeline([
    ("scaler", StandardScaler()),
    ("model", lgb.LGBMRegressor(objective="huber", random_state=42))
])

gkf = GroupKFold(n_splits=len(groups.unique()))
search_A = RandomizedSearchCV(
    pipe_A,
    param_distributions=param_space_A,
    n_iter=30,
    cv=gkf,
    scoring="neg_mean_squared_error",
    n_jobs=-1,
    random_state=42,
    verbose=1
)

# fit to find best Ea hyperparameters
search_A.fit(X_A, y_A, groups=groups)
best_params_A = search_A.best_params_
print(">> Best A‐model params:", best_params_A)

# leave-one-fuel-out evaluation of the tuned Ea model
A_metrics = []
df['A_pred_tuned'] = np.nan

for fuel in groups.unique():
    tr = groups != fuel
    te = ~tr

    Xtr, ytr = X_A[tr], y_A[tr]
    Xte, yte = X_A[te], y_A[te]

    # scale
    scaler = StandardScaler().fit(Xtr)
    Xtr_s, Xte_s = scaler.transform(Xtr), scaler.transform(Xte)

    tuned_A = lgb.LGBMRegressor(
        objective="huber",
        random_state=42,
        **{k.split("__",1)[1]: v for k, v in best_params_A.items()}
    )
    tuned_A.fit(Xtr_s, ytr)

    # predict & log metrics
    ypred = tuned_A.predict(Xte_s)
    A_metrics.append({
        "Fuel": fuel,
        "R2":   r2_score(yte, ypred),
        "MAE":  mean_absolute_error(yte, ypred),
        "MSE": mean_squared_error(yte, ypred),
    })

    # store predictions back in df
    df.loc[te, 'A_pred_tuned'] = ypred

scaler_A_all = StandardScaler().fit(X_A)
X_A_all_scaled = scaler_A_all.transform(X_A)

# Train model with best found hyperparameters
final_A_model = lgb.LGBMRegressor(
    objective="huber",
    random_state=42,
    **{k.split("__", 1)[1]: v for k, v in best_params_A.items()}
)
final_A_model.fit(X_A_all_scaled, y_A)

#joblib.dump((final_A_model, scaler_A_all), PATH HERE")

# display Ea tuning results
A_results_df = pd.DataFrame(A_metrics)
print("\nTuned Ea LOFO results:")
print(A_results_df)

# use the tuned predictions going forward
df['A_Arr'] = df['A_pred_tuned']

# ----------------------------------------------------------------------
# 3) PREPARE FOR STEP-2
# ----------------------------------------------------------------------
second = df.reset_index(drop=True)
groups = second['fuel_type']

drop2 = [
    'fuel_type','fuel_category','Unnamed: 0', 'E_A', 'A0', 'fuel_category2', 'thermal_treatment', 'A_pred_tuned', 'Ea_pred_tuned'
]
clean = second.drop(columns=drop2, errors='ignore').reset_index(drop=True)
clean['devol_yield'] = second['devol_yield']

# ----------------------------------------------------------------------
# 4) DEFINE MODELS & HYPERPARAMETER SPACES
# ----------------------------------------------------------------------
models = {
    "GB":  GradientBoostingRegressor,
    "XGB": xgb.XGBRegressor,
    "LGB": lgb.LGBMRegressor
}
param_space = {
    "GB": {
        "model__n_estimators":  randint(100,500),
        "model__learning_rate": uniform(0.05,0.2),
        "model__max_depth":     randint(3,7),
        "model__subsample":     uniform(0.7,0.3),
    },
    "XGB": {
        "model__n_estimators":     randint(100,500),
        "model__learning_rate":    uniform(0.05,0.2),
        "model__max_depth":        randint(3,7),
        "model__subsample":        uniform(0.7,0.3),
        "model__colsample_bytree": uniform(0.7,0.3),
    },
    "LGB": {
        "model__n_estimators":     randint(100,500),
        "model__learning_rate":    uniform(0.05,0.2),
        "model__num_leaves":       randint(20,80),
        "model__subsample":        uniform(0.7,0.3),
        "model__colsample_bytree": uniform(0.7,0.3),
    }
}

# ----------------------------------------------------------------------
# 5) DEFINE FEATURE-ENGINEERING CONFIGURATIONS
# ----------------------------------------------------------------------
# feature set definitions
baseline_feats = ['hc','oc', 'vm_fc' ,'temperature', 'heat_rate', 'residence_time', 'pressure']
baseline_Ea_feats = baseline_feats + ['EA_Arr']
baseline_Ea_A_feats = baseline_Ea_feats + ['A_Arr']
full_Ea_A = ['c', 'h', 'o', 'vm', 'fc','ac', 'n', 's' ,'cl', 'lhv',
           'hc','oc', 'temperature', 'pressure', 'heat_rate', 'residence_time', 'EA_Arr', 'A_Arr']
full_Ea = ['c', 'h', 'o', 'vm', 'fc','ac', 'n', 's' ,'cl', 'lhv',
           'hc','oc', 'temperature', 'pressure', 'heat_rate', 'residence_time', 'EA_Arr']
full = [f for f in full_Ea if f not in ['EA_Arr', 'A_Arr']]
ratio_feats    = ['vm_fc','t_to_T', 'ac_fc' ,'hc', 'oc','temperature', 'heat_rate', 'residence_time', 'pressure' ]
ratio_feats_Ea = ratio_feats + ['EA_Arr']
ratio_feats_Ea_A = ratio_feats_Ea + ['A_Arr']
operational_feats = ['temperature', 'heat_rate', 'residence_time', 'pressure']
ratio_feats_log = [f'log_{col}' for col in ['vm_fc', 'ac_fc', 'cl_ac', 'n_ac', 't_to_T','oc', 'hc', 'heat_rate', 'pressure']] + ['temperature', 'residence_time']
ratio_feats_log_Ea = ratio_feats_log + ['EA_Arr']
ratio_feats_log_Ea_A = ratio_feats_log_Ea + ['A_Arr']
ohe_feats2 = [c for c in clean.columns if c.startswith('th_treat')]
ohe_feats3 = [c for c in clean.columns if c.startswith('fuel_cat2')]


feature_sets = {
    "baseline":         baseline_feats,
    "baseline_ohe":     baseline_feats  + ohe_feats2 +ohe_feats3 ,
    "baseline_Ea":      baseline_Ea_feats,
    "baseline_Ea_ohe":  baseline_Ea_feats + ohe_feats2 +ohe_feats3 ,
    "baseline_Ea_A":    baseline_Ea_A_feats,
    "baseline_Ea_A_ohe": baseline_Ea_A_feats  + ohe_feats2 +ohe_feats3 ,
    "ratios":           ratio_feats,
    "ratios_ohe":       ratio_feats  + ohe_feats2 +ohe_feats3,
    "ratios_Ea":        ratio_feats_Ea,
    "ratios_Ea_ohe":    ratio_feats_Ea + ohe_feats2 +ohe_feats3 ,
    "ratios_Ea_A"  : ratio_feats_Ea_A,
    "ratios_Ea_A_ohe"  : ratio_feats_Ea_A  + ohe_feats2 +ohe_feats3,
    "full":             full,
    "full_Ea":          full_Ea,
    "full_Ea_ohe":      full_Ea  + ohe_feats2 +ohe_feats3 ,
    "full_Ea_A": full_Ea_A,
    "full_Ea_A_ohe": full_Ea_A+ ohe_feats2 +ohe_feats3,
    "full_ohe":         full + ohe_feats2 +ohe_feats3,
    "ratio_log":        ratio_feats_log,
    "ratio_log_Ea":     ratio_feats_log_Ea,
    "ratio_log_Ea_ohe": ratio_feats_log_Ea + ohe_feats2 +ohe_feats3,
    "ratio_log_Ea_A" : ratio_feats_log_Ea_A,
    "ratio_log_Ea_A_ohe" : ratio_feats_log_Ea_A + ohe_feats2 +ohe_feats3,

}

# ----------------------------------------------------------------------
# 6) LOOP OVER FEATURE SETS → TUNE & EVALUATE
# ----------------------------------------------------------------------
all_results    = []
all_parity     = []
best_params_all = {}

for fs_name, feats in feature_sets.items():
    print(f"\n=== Feature set: {fs_name} ===")
    Xc = clean[feats]
    yc = clean['devol_yield']
    best_params = {}
    gkf = GroupKFold(n_splits=len(groups.unique()))
    for name, Est in models.items():
        pipe = Pipeline([("scaler", StandardScaler()), ("model", Est())])
        search = RandomizedSearchCV(
            pipe,
            param_distributions=param_space[name],
            n_iter=20,
            cv=gkf,
            scoring="neg_mean_squared_error",
            n_jobs=-1,
            random_state=42
        )
        search.fit(Xc, yc, groups=groups)
        best_params[name] = search.best_params_
        print(f"  {name} best params → {search.best_params_}")
    best_params_all[fs_name] = best_params
    for fuel in groups.unique():
        tr = groups != fuel
        te = ~tr
        Xtr, ytr = Xc[tr], yc[tr]
        Xte, yte = Xc[te], yc[te]

        scaler = StandardScaler().fit(Xtr)
        Xtr_s, Xte_s = scaler.transform(Xtr), scaler.transform(Xte)

        for name, Est in models.items():
            params = {k.split("__",1)[1]: v for k,v in best_params[name].items()}
            m = Est(**params)
            m.fit(Xtr_s, ytr)
            yp = m.predict(Xte_s)

            all_results.append({
                "FeatureSet": fs_name,
                "Fuel":       fuel,
                "Model":      name,
                "R2":         r2_score(yte, yp),
                "MAE":        mean_absolute_error(yte, yp),
                "RMSE":       np.sqrt(mean_squared_error(yte, yp)),
            })
            for t_val, p_val in zip(yte, yp):
                all_parity.append({
                    "FeatureSet": fs_name,
                    "Fuel":       fuel,
                    "Model":      name,
                    "True":       t_val,
                    "Predicted":  p_val
                })


## Best performers list

In [ ]:
results_df = pd.DataFrame(all_results)
parity_df  = pd.DataFrame(all_parity)

print("\nResults (first 10 rows):")
print(results_df.head(10))
print("\nParity (first 10 rows):")
print(parity_df.head(10))

# ----------------------------------------------------------------------
# 8) IDENTIFY THE BEST MODEL FEATURE SET COMBINATION
# ----------------------------------------------------------------------

# Calculate macro-averaged MAE across fuels
avg_perf = (results_df.groupby(['FeatureSet', 'Model'])
                     .agg(avg_R2=('R2', 'mean'),
                          avg_MAE=('MAE', 'mean'),
                          avg_RMSE=('RMSE', 'mean'))
                     .reset_index())

# Choose model with the lowest avg MAE (you can also prioritize highest avg R2)
best_row = avg_perf.sort_values(by='avg_MAE').iloc[0]

final_fs_name     = best_row['FeatureSet']
final_model_name  = best_row['Model']
final_feats       = feature_sets[final_fs_name]

print(f"\nBest model: {final_model_name} with feature set: {final_fs_name}")
print(best_row)

# ----------------------------------------------------------------------
# 9) TRAIN FINAL MODEL ON FULL DATA (except validation)
# ----------------------------------------------------------------------

# Prepare full training data
X_final = clean[final_feats]
y_final = clean['devol_yield']
scaler_final = StandardScaler().fit(X_final)
X_final_scaled = scaler_final.transform(X_final)

# best parameters for the final model
best_params_final = {
    k.split("__", 1)[1]: v
    for k, v in best_params_all[final_fs_name][final_model_name].items()
}

# Instantiate the model
if final_model_name == "XGB":
    final_model = xgb.XGBRegressor(random_state=42, **best_params_final)
elif final_model_name == "LGB":
    final_model = lgb.LGBMRegressor(random_state=42, **best_params_final)
elif final_model_name == "GB":
    final_model = GradientBoostingRegressor(random_state=42, **best_params_final)
else:
    raise ValueError(f"Unknown model type: {final_model_name}")

# Fit and save
final_model.fit(X_final_scaled, y_final)

#joblib.dump((final_model, scaler_final),path)


In [ ]:

# Filter for best performing algorithm (XGB) results only
xgb_results = results_df[results_df['Model'] == 'XGB']

# Compute average performance across fuels for each feature set (only for XGB)
xgb_avg_perf = (xgb_results.groupby('FeatureSet')
                            .agg(avg_R2=('R2', 'mean'),
                                 avg_MAE=('MAE', 'mean'),
                                 avg_RMSE=('RMSE', 'mean'))
                            .reset_index())

# Sort by MAE to find best-performing feature set for XGB
best_xgb_row = xgb_avg_perf.sort_values(by='avg_MAE').iloc[1]

final_fs_name     = best_xgb_row['FeatureSet']
final_model_name  = 'XGB'
final_feats       = feature_sets[final_fs_name]

print(f"\nBest XGB feature set: {final_fs_name}")
print(best_xgb_row)


## SHAP Analysis

In [ ]:
import shap

X_final = clean[final_feats]
y_final = clean['devol_yield']

# Scale the features
scaler_final = StandardScaler().fit(X_final)
X_final_scaled = scaler_final.transform(X_final)
X_final_scaled_df = pd.DataFrame(X_final_scaled, columns=X_final.columns)

# ----------------------------------------------------------------------
# 2. Define prefixes of OHE features to exclude
# ----------------------------------------------------------------------
ohe_prefixes = ['fuel_cat2_', 'th_treat_']

# Filter out OHE features
non_ohe_features = [
    col for col in X_final_scaled_df.columns
    if not any(col.startswith(prefix) for prefix in ohe_prefixes)
]

# Get column indices for SHAP filtering
non_ohe_indices = [X_final_scaled_df.columns.get_loc(col) for col in non_ohe_features]

# ----------------------------------------------------------------------
# 3. SHAP explanation using TreeExplainer
# ----------------------------------------------------------------------
explainer = shap.Explainer(final_model, X_final_scaled_df)
shap_values = explainer(X_final_scaled_df)

# Filter SHAP values to exclude OHE columns
shap_values_filtered = shap_values[:, non_ohe_indices]
X_filtered_df = X_final_scaled_df[non_ohe_features]

shap.plots.beeswarm(shap_values_filtered, max_display=20)
plt.tight_layout()
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()




In [ ]:
import matplotlib.pyplot as plt
metrics = ['R2', 'MAE', 'RMSE']
feature_sets = results_df['FeatureSet'].unique()
models_list  = results_df['Model'].unique()

for metric in metrics:
    plt.figure(figsize=(20, 8))
    data = []
    labels = []
    for fs in feature_sets:
        for model in models_list:
            vals = results_df[
                (results_df['FeatureSet'] == fs) &
                (results_df['Model'] == model)
            ][metric]
            data.append(vals)
            labels.append(f"{fs}-{model}")
    plt.boxplot(data, labels=labels)
    plt.xticks(rotation=45, ha='right')
    plt.title(f"{metric} Distribution by FeatureSet & Model")
    plt.ylabel(metric)
    plt.tight_layout()
    plt.show()
fuels = parity_df['Fuel'].unique()

perf_lookup = (
    results_df
    .set_index(['FeatureSet','Model','Fuel'])
    [['R2','MAE','RMSE']]
)

In [ ]:

perf_lookup = results_df.set_index(['FeatureSet','Model','Fuel'])[['R2','MAE','RMSE']]

feature_sets = results_df['FeatureSet'].unique()
models_list  = results_df['Model'].unique()
fuels_list   = results_df['Fuel'].unique()

cmap = plt.cm.get_cmap('tab20', len(fuels_list))
fuel_colors = {fuel: cmap(i) for i, fuel in enumerate(fuels_list)}

for fs in feature_sets:
    for model in models_list:
        sub = parity_df[(parity_df['FeatureSet']==fs) & (parity_df['Model']==model)]
        if sub.empty:
            continue

        # grab best params for this (fs, model)
        bp = best_params_all[fs][model]
        param_str = ", ".join(f"{k.split('__',1)[1]}={v}" for k,v in bp.items())

        plt.figure(figsize=(12,8))
        for fuel in fuels_list:
            sub2 = sub[sub['Fuel']==fuel]
            if sub2.empty:
                continue

            r2, mae, rmse = perf_lookup.loc[(fs, model, fuel)]
            label = f"{fuel}\nR²={r2:.2f}, MAE={mae:.2f}, RMSE={rmse:.2f}"

            plt.scatter(sub2['True'], sub2['Predicted'],
                        label=label, alpha=0.6, s=60,
                        color=fuel_colors[fuel])

        mn, mx = sub[['True','Predicted']].values.min(), sub[['True','Predicted']].values.max()
        plt.plot([mn,mx],[mn,mx], '--', color='gray')

        plt.title(f"{model} | {fs}\nBest params: {param_str}", fontsize=14)
        plt.xlabel('True Volatile Yield', fontsize=14)
        plt.ylabel('Predicted Volatile Yield', fontsize=14)
        plt.legend(bbox_to_anchor=(1.05,1), loc='upper left', fontsize='large')
        plt.tight_layout()
        plt.show()


In [ ]:
metrics      = ['R2', 'MAE', 'RMSE']
feature_sets = results_df['FeatureSet'].unique()
models_list  = results_df['Model'].unique()

N = len(feature_sets)
angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
angles += angles[:1]
cmap = plt.get_cmap('tab10')
model_colors = {model: cmap(i) for i, model in enumerate(models_list)}

fig, axes = plt.subplots(
    1, 3,
    figsize=(22, 8),
    subplot_kw=dict(polar=True)
)
fig.suptitle('Model Comparison Across Feature Sets', fontsize=24, y=1)

handles = []
labels = []

for ax, metric in zip(axes, metrics):
    for model in models_list:
        df_m = results_df[results_df['Model'] == model]
        agg  = df_m.groupby('FeatureSet')[metric].mean()
        values = [agg.get(fs, np.nan) for fs in feature_sets]
        values += values[:1]

        h, = ax.plot(angles, values,
                     color=model_colors[model],
                     linewidth=2,
                     label=model)
        ax.fill(angles, values,
                color=model_colors[model],
                alpha=0.15)

        if metric == metrics[0]:
            handles.append(h)
            labels.append(model)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(feature_sets, fontsize=12)
    ax.set_title(f'Mean {metric}', pad=15, fontsize=18)

    if metric == 'R2':
        ax.set_ylim(0, 1)
    else:
        overall_max = results_df.groupby(['Model','FeatureSet'])[metric].mean().max()
        ax.set_ylim(0, overall_max * 1.1)

legend = fig.legend(
    handles,
    labels,
    loc='upper center',
    ncol=len(models_list),
    bbox_to_anchor=(0.5, 0.02),
    prop={'size': 18},
    handletextpad=1.5,
    handlelength=3,
    columnspacing=2)

legend.get_frame().set_linewidth(1.5)
legend.get_frame().set_edgecolor('black')

plt.tight_layout(rect=[0, 0.05, 1, 0.95])
plt.show()


In [ ]:
from itertools import compress

all_cols      = results_df.columns.tolist()
fixed_cols    = ['FeatureSet','Model','Fuel','R2','MAE','RMSE']
hp_cols       = [c for c in all_cols if c not in fixed_cols]

# Aggregate performance metrics across fuels
agg = (
    results_df
    .groupby(['FeatureSet','Model'] + hp_cols)
    .agg({
        'R2':  'mean',
        'MAE': 'mean',
        'RMSE':'mean'
    })
    .reset_index()
)

# Sort to get the “best” at the top
best = agg.sort_values(
    by=['R2','MAE','RMSE'],
    ascending=[False, True, True]
).reset_index(drop=True)

print("Top 10 hyper‐parameter combos (averaged over all fuels):")
print(best.head(10))

is_dominated = [False]*len(best)
for i, row_i in best.iterrows():
    for j, row_j in best.iterrows():
        if (row_j['R2']   >= row_i['R2'] and
            row_j['MAE']  <= row_i['MAE'] and
            row_j['RMSE'] <= row_i['RMSE']
           ) and (j != i):

            if ((row_j['R2']   > row_i['R2']) or
                (row_j['MAE']  < row_i['MAE']) or
                (row_j['RMSE'] < row_i['RMSE'])):
                is_dominated[i] = True
                break

pareto = best.loc[list(compress(range(len(best)), [not d for d in is_dominated]))]
print("\nPareto‐optimal hyper‐parameter settings:")
print(pareto)


## Varience Inflation Factor
Change over feature sets

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

# ----------------------------------------------------------------------
# VIF DETERMINATION ACROSS FEATURE SETS
# ----------------------------------------------------------------------

baseline_feats = ['hc','oc', 'vm_fc' ,'temperature', 'heat_rate', 'residence_time', 'pressure']
baseline_Ea_feats = baseline_feats + ['EA_Arr']
baseline_Ea_A_feats = baseline_Ea_feats + ['A_Arr']
full_Ea_A = ['c', 'h', 'o', 'vm', 'fc','ac', 'n', 's' ,'cl', 'lhv',
           'hc','oc', 'temperature', 'pressure', 'heat_rate', 'residence_time', 'EA_Arr', 'A_Arr']
full_Ea = ['c', 'h', 'o', 'vm', 'fc','ac', 'n', 's' ,'cl', 'lhv',
           'hc','oc', 'temperature', 'pressure', 'heat_rate', 'residence_time', 'EA_Arr']
full = [f for f in full_Ea if f not in ['EA_Arr', 'A_Arr']]
ratio_feats    = ['vm_fc','t_to_T', 'ac_fc' ,'hc', 'oc','temperature', 'heat_rate', 'residence_time', 'pressure' ]
ratio_feats_Ea = ratio_feats + ['EA_Arr']
ratio_feats_Ea_A = ratio_feats_Ea + ['A_Arr']
operational_feats = ['temperature', 'heat_rate', 'residence_time', 'pressure']
ratio_feats_log = [f'log_{col}' for col in ['vm_fc', 'ac_fc', 'cl_ac', 'n_ac', 't_to_T','oc', 'hc', 'heat_rate', 'pressure']] + ['temperature', 'residence_time']
ratio_feats_log_Ea = ratio_feats_log + ['EA_Arr']
ratio_feats_log_Ea_A = ratio_feats_log_Ea + ['A_Arr']
ohe_feats2 = [c for c in clean.columns if c.startswith('th_treat')]
ohe_feats3 = [c for c in clean.columns if c.startswith('fuel_cat2')]


feature_sets = {
    "baseline":         baseline_feats,
    "baseline_ohe":     baseline_feats  + ohe_feats2 +ohe_feats3 ,
    "baseline_Ea":      baseline_Ea_feats,
    "baseline_Ea_ohe":  baseline_Ea_feats + ohe_feats2 +ohe_feats3 ,
    "baseline_Ea_A":    baseline_Ea_A_feats,
    "baseline_Ea_A_ohe": baseline_Ea_A_feats  + ohe_feats2 +ohe_feats3 ,
    "ratios":           ratio_feats,
    "ratios_ohe":       ratio_feats  + ohe_feats2 +ohe_feats3,
    "ratios_Ea":        ratio_feats_Ea,
    "ratios_Ea_ohe":    ratio_feats_Ea + ohe_feats2 +ohe_feats3 ,
    "ratios_Ea_A"  : ratio_feats_Ea_A,
    "ratios_Ea_A_ohe"  : ratio_feats_Ea_A  + ohe_feats2 +ohe_feats3,
    "full":             full,
    "full_Ea":          full_Ea,
    "full_Ea_ohe":      full_Ea  + ohe_feats2 +ohe_feats3 ,
    "full_Ea_A": full_Ea_A,
    "full_Ea_A_ohe": full_Ea_A+ ohe_feats2 +ohe_feats3,
    "full_ohe":         full + ohe_feats2 +ohe_feats3,
    "ratio_log":        ratio_feats_log,
    "ratio_log_Ea":     ratio_feats_log_Ea,
    "ratio_log_Ea_ohe": ratio_feats_log_Ea + ohe_feats2 +ohe_feats3,
    "ratio_log_Ea_A" : ratio_feats_log_Ea_A,
    "ratio_log_Ea_A_ohe" : ratio_feats_log_Ea_A + ohe_feats2 +ohe_feats3,

}

for fs_name, feats in feature_sets.items():
    df_sub = clean[feats].dropna()
    df_sub = df_sub.select_dtypes(include=[np.number])
    df_sub = df_sub.loc[:, df_sub.nunique() > 1]
    if df_sub.shape[1] < 2:
        print(f"[{fs_name}] only {df_sub.shape[1]} varying feature(s), skipping.\n")
        continue
    X = df_sub.values.astype(float)

    vif_results = []
    for i, col in enumerate(df_sub.columns):
        vif = variance_inflation_factor(X, i)
        vif_results.append((col, vif))

    vif_df = (
        pd.DataFrame(vif_results, columns=["Feature", "VIF"])
          .sort_values("VIF", ascending=False)
          .reset_index(drop=True)
    )
    print(f"\n=== VIF for feature-set: {fs_name} (no intercept) ===")
    print(vif_df.to_string(index=False))


In [ ]:
perf_lookup  = results_df.set_index(['FeatureSet','Model','Fuel'])[['R2','MAE','RMSE']]
fuels_list   = results_df['Fuel'].unique()
cmap         = plt.cm.get_cmap('tab20', len(fuels_list))
fuel_colors  = {fuel: cmap(i) for i, fuel in enumerate(fuels_list)}

# Top-6 combos PARITY
top6 = best.head(6)

fig, axes = plt.subplots(3, 2, figsize=(20, 28))
axes = axes.flatten()
for ax, (_, row) in zip(axes, top6.iterrows()):
    fs, model = row['FeatureSet'], row['Model']
    bp = best_params_all[fs][model]

    parts = []
    for key, val in bp.items():
        name = key.split('__',1)[1]
        parts.append(f"{name}={val:.4f}" if isinstance(val, float) else f"{name}={val}")
    param_str = ", ".join(parts)
    sub = parity_df[(parity_df['FeatureSet']==fs) & (parity_df['Model']==model)]
    if sub.empty:
        continue

    for fuel in fuels_list:
        sub2 = sub[sub['Fuel']==fuel]
        if sub2.empty:
            continue
        r2, mae, rmse = perf_lookup.loc[(fs,model,fuel)]
        lbl = f"{fuel} (R²={r2:.2f}, MAE={mae:.2f}, RMSE={rmse:.2f})"
        ax.scatter(sub2['True'], sub2['Predicted'],
                   color=fuel_colors[fuel], alpha=0.6, label=lbl)

    mn, mx = sub[['True','Predicted']].values.min(), sub[['True','Predicted']].values.max()
    ax.plot([mn,mx],[mn,mx],'--',color='gray')

    ax.set_title(f"{model} | {fs}\nBest params: {param_str}", fontsize=12)
    ax.set_xlabel('True Volatile Yield')
    ax.set_ylabel('Predicted Volatile Yield')

    ax.legend(
        loc='lower center',
        bbox_to_anchor=(0.5, -0.45),
        bbox_transform=ax.transAxes,
        ncol=2,
        fontsize='medium',
        frameon=True,
        borderpad=0.5
    )

plt.subplots_adjust(hspace=0.6, wspace=0.27)
plt.show()
